In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

tf.reset_default_graph()
logdir = 'C:/code/log'

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [ ]:

# 【新增】 准备一个用于保存训练和恢复的saver
import os 
ckpt_dir = './ckpt_dir/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
    

#输入层 输出层
x = tf.placeholder(tf.float32, [None, 784], name="X")
y = tf.placeholder(tf.float32, [None, 10], name="Y")


image_shaped_input = tf.reshape(x,[-1,28,28,1])
tf.summary.image('input',image_shaped_input,10)

# 神经元个数
H1_NN = 256
H2_NN = 64
H3_NN = 32

def fcn_layer(inputs, # 输入数据
              input_dim, # 神经元数量
              output_dim, #输出神经元数量
              activation = None # 激活函数
             ):
    W = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1))
    b = tf.Variable(tf.zeros([output_dim]))
    
    XWb = tf.matmul(inputs, W) + b
    
    if activation is None:
        outputs = XWb
    else:
        outputs = activation(XWb)
        
    return outputs

h1 = fcn_layer(x, 784, H1_NN, tf.nn.relu )
h2 = fcn_layer(h1, H1_NN, H2_NN, tf.nn.relu )
h3 = fcn_layer(h2, H2_NN, H3_NN, tf.nn.relu )
forward = fcn_layer(h3, H3_NN, 10)

# 直方图显示
tf.summary.histogram('forward', forward)

# 直接用softmax 加交叉熵会loss爆炸，因为有log(0)的计算
pred = tf.nn.softmax(forward)
#loss_function = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))

# v2 用softmax_cross_entropy_with_logits api 会处理交叉熵的这个问题
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=forward,labels = y))

# 标量显示loss
tf.summary.scalar('loss', loss_function)

#训练参数
train_epochs = 5
batch_size = 50
total_batch = int(mnist.train.num_examples/batch_size)
display_step = 1
learning_rate = 0.01

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [ ]:
# 【新增】存储模型的粒度
save_step = 5
# 所有变量声明完之后，调用tf.train.Saver()
saver = tf.train.Saver()


from time import time
startTime = time()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

merged_summary_op = tf.summary.merge_all()
writer = tf.summary.FileWriter('log/', sess.graph)
for epoch in range(train_epochs):
    for batch in range(total_batch):
        xs, ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x:xs, y:ys})
        
        # 生成summary
        summary_str = sess.run(merged_summary_op, feed_dict={x:xs,y:ys})
        # summary写入文件
        writer.add_summary(summary_str,epoch) 
        
    loss, acc = sess.run([loss_function, accuracy], feed_dict={
        x:mnist.validation.images, y:mnist.validation.labels
    })
    
    if(epoch+1)% display_step == 0:
        print("epoch = " , epoch+1,' loss = ', loss, " accuracy = ", acc)
        
    # 【新增】
    if(epoch+1)% save_step == 0: 
            saver.save(sess, os.path.join(ckpt_dir, 'mnist_h256_model_{:06d}.ckpt'.format(epoch+1)))
            
saver.save(sess, os.path.join(ckpt_dir, 'mnist_h256_model.ckpt'))
print("model saved")
            
duration = time() - startTime
print('Train finished takes : ' , duration)

epoch =  1  loss =  0.189344  accuracy =  0.9526
epoch =  2  loss =  0.137486  accuracy =  0.9648


In [ ]:
accu_test = sess.run(accuracy, feed_dict={
    x: mnist.test.images, y:mnist.test.labels
})

print('test accuracy', accu_test)


In [ ]:
# one_hot to 数字（index）
prediction_result = sess.run(tf.argmax(pred, 1),feed_dict={x:mnist.test.images})
# prediction_result[0:10]

import numpy as np
compare_list = prediction_result == np.argmax(mnist.test.labels, 1)
#print(compare_list)
err_list = [i for i in range(len(compare_list)) if compare_list[i]==False]
#print(err_list)
#print(len(err_list))

In [ ]:
def print_predict_err(labels, prediction):
    count = 0 
    compare_list = (prediction == np.argmax(labels, 1))
    err_list = [i for i in range(len(compare_lists)) if compare_list[i] == False]
    for x in err_lists:
        print("index = ", x, " 标签值 = ", np.argmax(labels[x]), " 预测值 = ", prediction[x] )
        count = count + 1
    print ("总计 : ",count )

In [ ]:
import matplotlib.pyplot as plt
# 可视化看结果的函数，最多显示25张图
def plot_images_labels_prediction(images,
                                  labels,
                                  prediction,
                                  index, # 从第index个开始显示
                                  num=10):
    fig = plt.gcf()# get current figure
    fig.set_size_inches(10,12) # 当前图像大小为10英寸*12英寸
    if num>25:
        num = 25
    for i in range(0,num):
        ax = plt.subplot(5,5,i+1) # 获取当前要处理的子图
        ax.imshow(np.reshape(images[index],(28,28)),cmap='binary') #指定image reshape
        title = 'label=' + str(np.argmax(labels[index]))
        
        ax.set_title(title,fontsize=10) # 在图上显示预测值
        ax.set_xticks([]) # 为了美观， 不显示x和y轴
        ax.set_xticks([])
        index = index + 1
    plt.show()

plot_images_labels_prediction(mnist.test.images, mnist.test.labels, prediction_result, 610, 20)

In [ ]:
writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
writer.close()